 Домашна работа по предметот Методологија на истражувањето во икт
Индекс: 143082
Име: Викторија
Презиме: Голужа
За да филтрираме слика во фреквенциски домен потребно е да ја помножиме сликата во фреквенциски домен со филтерот (кернел) кој е исто така во фреквенциски домен.
$$ I_{f,filt} = HI_f $$

За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:
$$ H(i,j) = e^{\frac{d(i,j)^2}{f_h^2}} (1 - e^{\frac{d(i,j)^2}{f_l^2}}) $$
Каде што 𝑑(𝑖,𝑗) е евклидовото растојанието на даден пиксел од центарот на сликата, додека 𝑓ℎ и 𝑓𝑙 се праговите на високите и ниските фреквенции кои сакаме
да ги филтрираме.

Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација. Фуриевата трансформација е дефинирана
како:

$$ F(u,v) = \iint \limits_{-\infty}^{\infty} f(x,y)e^{-i2\pi(ux+vy)}dxdy   $$
А во дискретниот домен горната равенка може да биде преведена како:
$$ F(u,v) = \mathop{\sum_{m=-\infty}^{\infty}\sum_{n=-\infty}^{\infty}} f[m,n]\cdot e^{-i2\pi(ux+vy)}dudv $$
Конверзија на 2D сликата од фреквенциски во просторен домен се прави со користење на инверзна Фуриева трансформација. Инверзната Фуриева
трансформација е дефинирана како:

$$ F(x,y) = \iint \limits_{-\infty}^{\infty} f(u,v) e^{-i2\pi(ux+vy)}dudv   $$
А во дискретниот домен горната равенка може да биде преведена како:
$$ F(x,y) = \mathop{\sum_{m=-\infty}^{\infty}\sum_{n=-\infty}^{\infty}} f[m,n] \cdot e^{-i2\pi(xmu_0+ynv_0)} $$
Целите на оваа домашна е се искористат Фуриеви Трансформации на слики користејки го OpenCV - package во JupyterNotebook и да можеме да искористиме FastFourierTransform(FFT). Ке ги искористиме 2те функции кои ги нуди пакетот OpenCV , односно cv.dft() и cv.idft()

Фуриевите Трансформации се користат за да се анализираат фрекфентните карактеристики на различни филтри. За слики , се користи 2D DFT и таа се користи за да се најдат фрекфентните домени. FFT алгоритамот се користи за калкулациите што ги користи DFT(cv.dft()).
Ја користиме оваа формула за Дискретна Фуриева трансформација-DFT(Discrete Fourier Transform)
$$ Ak = \sum\limits<em>{m=0}^{n-1}a</em>{m}exp \left{ -2 \pi \frac{mk}{n} \right} k=0,....,n-1 $$

Ја користиме оваа формула за Инверзна Дискретна Фуриева трансформација- IDFT(Inverse Discrete Fourier Transform)
$$ Ak = {1 \over n} \sum\limits<em>{m=0}^{n-1}a</em>{m}exp \left{ -2 \pi \frac{mk}{n} \right} m=0,....,n-1 $$

Фуриеви Трансформации во Numpy
Најпрвин користиме FFT што ја поседува пакетот numpy односно np.fft.fft2() таа не снабдува со фрекфенциите кои што се во некоја низа. Функцијата прима како аргумент слика која што мора да бие во grayscale. Вториот аргумент е опционален и тоа е колкава да биде низата. Ако низата е поголема од големината на сликата таа ке биде наполнета со 0, во обратен случан сликата ќе биде скратена до големината на сликата.

Одкако ќе го добиеме резултатот , DC компонетната ке ни биде во левиот ќош од низата, за да го донесеме во средина користиме np.fft.fftshift(). Одкако ја пронаоѓаме фрекфенцијата, можеме да го пронајдеме магнитудниот спектрум.

In [68]:
import numpy as np
import cv2
import math
from matplotlib import pyplot as plt
path = r'D:\downloads\Domasna_rabota_metodologija\Barbara.tif'

def read_image(path):
    img = cv2.imread(path,0) 
    return img

img = read_image(path)
# print(img_fft)
no_rows = img.shape[0]
no_cols = img.shape[1]
# print(no_rows)

[[ 518.        +1.98951966e-13j   33.2405986 -5.05746372e+02j
  -276.54222034+3.26977181e+02j ...   -4.87288438-9.29002608e+00j
  -276.54222034-3.26977181e+02j   33.2405986 +5.05746372e+02j]
 [ 611.89230178-3.55440764e+02j  649.30918323+9.83903570e+02j
    62.4911446 +3.24816539e+02j ...  391.12886214-7.99146552e+02j
   297.28828234+3.94874883e+02j  152.37937673-6.59912185e+02j]
 [ 408.07742883-1.31292891e+02j -589.52155475+3.48702369e+01j
  -670.16616866+6.34281926e+02j ... 1183.04117109+7.91606984e+02j
   633.90027362-5.21066791e+02j  -52.78849789+7.14286819e+02j]
 ...
 [ 290.12152403+3.34732392e+02j  380.6889522 +4.51925173e+02j
  -652.73601475-2.92095826e+02j ... -197.19669399-6.31871666e+02j
   235.04476317+3.83750989e+02j -217.39103914-5.57199039e+02j]
 [ 408.07742883+1.31292891e+02j  -52.78849789-7.14286819e+02j
   633.90027362+5.21066791e+02j ... -184.06941725+2.93544399e+02j
  -670.16616866-6.34281926e+02j -589.52155475-3.48702369e+01j]
 [ 611.89230178+3.55440764e+02j  152.379

In [6]:
def create_filter(no_rows, no_cols, d0, d1):
    filtered_image = np.asarray(np.ones((no_rows, no_cols), dtype=np.float32))
    for i in range(0, no_rows-1):
        for j in range(0, no_cols-1):
#             print(i, j)
            dist = math.sqrt(math.pow(i - int(no_rows/2), 2) + math.pow(j - int(no_cols/2), 2))
            filtered_image[i+1,j+1] = math.exp(-math.pow(dist, 2)/(2*(math.pow(d1,2))))*(1.0-math.exp(-math.pow(dist, 2)/(2*(math.pow(d0, 2)))))

    filtered_image[int(no_rows/2)+1,int(no_cols/2)+1]=1.0
#     print(filtered_image[int(no_rows/2)+1,int(no_cols/2)+1])
    return filtered_image

In [107]:
def im2double(im):
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out

def get_filtered_image(img, l, h):
    fl = 1 + l*10
    fh = 1 + h*10
    
    img_double = img
    (nx, ny) = img_double.shape;
    img_fft = np.fft.fft(img_double)
    img_fft = np.fft.fftshift(np.fft.fft2(img_double));
    result_create_filter = create_filter(nx, ny, fl, fh)
    img_ifftshift = np.fft.ifftshift(img_fft*result_create_filter)
    img_ifft2 = np.fft.ifft2(img_ifftshift)
    result = abs(img_ifft2)
    return result

def get_filter(img, l, h):
    fl = 1 + l*10
    fh = 1 + h*10
    img_double = im2double(img);
    (nx, ny) = img_double.shape;
    result_create_filter = create_filter(nx, ny, fl, fh)
    return result_create_filter

from ipywidgets import interactive, widgets, interact, HBox, Label
from IPython.display import Javascript

def display_images(img, l, h):
    filter_img = get_filter(img, l, h)
#     print(filter_img)
    filtered_image = get_filtered_image(img, l, h)
#     vis2 = cv2.cvtColor(filter_img, cv2.COLOR_GRAY2BGR)
#     cv2.imshow('l' , vis2 )
    plt.imshow(filter_img, cmap='gray')
    plt.show()
    plt.imshow(filtered_image, cmap='gray')
    plt.show()

In [108]:

@interact(x=widgets.IntRangeSlider(
    value=[0, 10],
    min=0., max=10., step=1,
    description='Frequency cuttofs:',
))

def g(x):
    l=x[0]
    h=x[1]
    display_images(img, l, h)

interactive(children=(IntRangeSlider(value=(0, 10), description='Frequency cuttofs:', max=10), Output()), _dom…